# Instructions

- Copy all loki result files into **Loki_Files** folders
- Only Loki result should exist in the **Loki_Files** folder
- Result files must be generated by loki with option --csv 




In [ ]:
import glob
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
files = glob.glob(".\Loki_Files\*")
lines = []
for f in files:
    with open(f, encoding='utf-8') as fh:
        for line in fh:
            lines.append(line)


## Number of files

In [ ]:
len(files)

In [ ]:
cols = ['Hostname','Type', 'File',"Rule Name","Description"]
df = pd.DataFrame(columns = cols)
row = {}
pd.set_option('display.max_colwidth', None)

for line in lines:
    try:
        row['Hostname'] = line.split(',')[1]
    except IndexError:
        continue
        
    if "ALERT" in line or "WARNING" in line:
        if "ALERT" in line:
            row['Type'] = "Alert"  
        elif "WARNING" in line:
            row['Type'] = "Warning"
    else:
        row['Type'] = np.NAN
    match = re.search('FILE:\s(.*?)\sSCORE',line)
    if match:
        row['File'] = re.search('FILE:\s(.*?)\sSCORE',line).group(1)
            
    match = re.search('Yara Rule MATCH:\s(.*?)\s',line)   
    if match:
        row['Rule Name']= re.search('Yara Rule MATCH:\s(.*?)\s',line).group(1)
    match = re.search('DESCRIPTION:\s(.*?):',line)
    if match:
        row['Description']= re.search('DESCRIPTION:\s(.*?)REF:',line).group(1)
      
    df=df.append(row, ignore_index=True)
    


## Number of systems


In [ ]:
df['Hostname'].nunique()

## Number of systems with alerts or warnings

In [ ]:
filt = (df['Type'] == 'Alert' ) | (df['Type'] == 'Warning')
idf=df[filt]
idf['Hostname'].nunique()

# Number of warning and alert 

In [ ]:
idf["Type"].value_counts()

## Rule Names 

In [ ]:
idf['Rule Name'].value_counts()

# Top Infected Systems

In [ ]:
idf['Hostname'].value_counts()

# Top matching files

In [ ]:
idf['File'].value_counts().head(20)

In [ ]:
idf.groupby(["Hostname", "Rule Name"]).size()

## All Alerts

In [ ]:
idf[idf['Type'] =='Alert'].head(20)

## All Warnings

In [ ]:
idf[idf['Type']=="Warning"].head(20)